In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1. InVitroWildfire EVProteomics/P1.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1. InVitroWildfire EVProteomics/P1.3. Analyses/P1.3.1. PDF Data Extraction/Output')
cur_date = "011222"

library(tidyverse)
# library(reshape2)
library(pdftools)
library(tm)

#only grabs files ending in "pdf"
pdf_list = list.files(pattern = "pdf$")
head(pdf_list)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Using poppler version 20.12.1

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




[1] "20211018_0002_9.2.1_84_200_size.pdf"  
[2] "20211018_0003_9.2.1_95_200_size.pdf"  
[3] "20211018_0004_9.2.1_101_200_size.pdf" 
[4] "20211018_0005_9.2.1_109_2000_size.pdf"
[5] "20211018_0006_9.2.1_124_2000_size.pdf"
[6] "20211018_0007_9.2.1_125_2000_size.pdf"

# Data we want to extract:

1. Pdf file name as the sample identifier
2. Original Concentration
3. Some of the ‘X Values’ table listed on the bottom left, specifically the: X10 Concentration, X50 Concentration, X90 Concentration, and Mean Concentration, and StdDev Concentration

In [2]:
corpus_raw <- data.frame("Sample Identifier" = c(),"Text" = c())

for (i in 1:length(pdf_list)){
    document_text = pdf_text(paste(pdf_list[i],sep = "")) %>% 
    strsplit("\n")
    
        document = data.frame("Sample Identifier" = gsub(x =pdf_list[i],pattern = ".pdf", replacement = ""), 
        "Text" = document_text, stringsAsFactors = FALSE)
    
    colnames(document) <- c("Sample Identifier", "Text")
    corpus_raw <- rbind(corpus_raw,document) 
}

#still haven't figured out why there are duplicates
corpus_raw = unique(corpus_raw)

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

In [3]:
head(corpus_raw)

,Sample Identifier,Text
,<chr>,<chr>
1,20211018_0002_9.2.1_84_200_size,Electrophoresis & Brownian Motion
2,20211018_0002_9.2.1_84_200_size,Video Analysis
3,20211018_0002_9.2.1_84_200_size,Laser Scattering Microscopy
4,20211018_0002_9.2.1_84_200_size,www.particle-metrix.de
5,20211018_0002_9.2.1_84_200_size,
6,20211018_0002_9.2.1_84_200_size,Operator (Report): sop_kabanovlab.inst


In [4]:
OG_concentration_df = corpus_raw %>%
    filter(grepl("Original Concentration", Text)) %>%
    separate(Text, c(NA, "Original Concentration"), sep = ": ") %>%
    separate(`Original Concentration`, c("Original Concentration (Particles/ mL)", NA), sep = "P")
head(OG_concentration_df)

,Sample Identifier,Original Concentration (Particles/ mL)
,<chr>,<chr>
1,20211018_0002_9.2.1_84_200_size,4.7E+9
2,20211018_0003_9.2.1_95_200_size,6.5E+9
3,20211018_0004_9.2.1_101_200_size,6.7E+9
4,20211018_0005_9.2.1_109_2000_size,1.5E+11
5,20211018_0006_9.2.1_124_2000_size,1.6E+11
6,20211018_0007_9.2.1_125_2000_size,2.6E+11


In [5]:
X_concentration_df = corpus_raw %>%
    filter(grepl("X10", Text)| grepl("X50 ", Text)| grepl("X90", Text) | grepl("Mean", Text)| grepl("StdDev", Text)) %>%
    separate(Text, c("Text", NA), sep = "                ") %>%
    separate(Text, c("Text", "Concentration"), sep = "      ") %>%
    pivot_wider(names_from = Text, values_from = Concentration)
head(X_concentration_df)

Sample Identifier,X10,X50,X90,Mean,StdDev
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
20211018_0002_9.2.1_84_200_size,59.2,123.3,276.6,180.9,180.7
20211018_0003_9.2.1_95_200_size,60.0,145.0,310.0,196.9,161.7
20211018_0004_9.2.1_101_200_size,64.6,127.0,248.0,170.8,128.8
20211018_0005_9.2.1_109_2000_size,58.5,119.6,235.0,154.9,101.2
20211018_0006_9.2.1_124_2000_size,67.3,123.5,226.0,157.2,100.5
20211018_0007_9.2.1_125_2000_size,74.5,123.3,206.1,149.0,66.4


In [6]:
#combining into one final df
final_df = full_join(OG_concentration_df, X_concentration_df)
head(final_df)

Joining, by = "Sample Identifier"



,Sample Identifier,Original Concentration (Particles/ mL),X10,X50,X90,Mean,StdDev
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,20211018_0002_9.2.1_84_200_size,4.7E+9,59.2,123.3,276.6,180.9,180.7
2,20211018_0003_9.2.1_95_200_size,6.5E+9,60.0,145.0,310.0,196.9,161.7
3,20211018_0004_9.2.1_101_200_size,6.7E+9,64.6,127.0,248.0,170.8,128.8
4,20211018_0005_9.2.1_109_2000_size,1.5E+11,58.5,119.6,235.0,154.9,101.2
5,20211018_0006_9.2.1_124_2000_size,1.6E+11,67.3,123.5,226.0,157.2,100.5
6,20211018_0007_9.2.1_125_2000_size,2.6E+11,74.5,123.3,206.1,149.0,66.4


In [7]:
#exporting data
write.csv(final_df, paste0(Output,"/", cur_date, "_Final_PDF_Data.csv"), row.names = FALSE)